# Préparation des données de maintenance

Chargement des donnnées

In [1]:
import pandas as pd
import numpy  as np

#Chargement des données
maintenance = pd.read_csv("../data/raw/PdM_maint.csv")
# transformer les dates dans le bon type
maintenance["datetime"] = pd.to_datetime(maintenance["datetime"])

#Création des indexes
maintenance = maintenance.set_index(['machineID', 'datetime'])
maintenance = pd.get_dummies(maintenance["comp"])
maintenance = maintenance.groupby(["machineID","datetime"]).sum()

In [2]:
maintenance.tail(10)

comp1  comp2  comp3  comp4
machineID datetime                                       
100       2015-07-27 06:00:00      1      1      0      0
          2015-08-11 06:00:00      0      0      1      0
          2015-08-26 06:00:00      0      0      0      1
          2015-09-10 06:00:00      1      0      0      0
          2015-09-25 06:00:00      1      0      0      0
          2015-10-10 06:00:00      1      0      1      0
          2015-10-25 06:00:00      0      0      0      1
          2015-11-09 06:00:00      0      0      0      1
          2015-12-09 06:00:00      0      1      0      0
          2015-12-24 06:00:00      0      1      0      0

Pour chaque  composant, on calcule la durée depuis  le dernier changement

In [3]:
machines = pd.read_csv("../data/raw/PdM_machines.csv")

time_series = pd.date_range(start = "2014-06-01 06:00:00", end = "2016-01-01 06:00:00", freq="1D")
indexes = pd.MultiIndex.from_product([machines["machineID"].values,time_series.values], names=['machineID', 'datetime'])

In [4]:
maintenance = pd.DataFrame(index = indexes).join(maintenance).fillna(0)

In [5]:
days_since_maintenance1 = []
days_since_maintenance2 = []
days_since_maintenance3 = []
days_since_maintenance4 = []
for machine in machines["machineID"].values:
    days_count = 0
    for reset in maintenance.loc[machine]["comp1"]:
        days_count = (days_count+1)*(1-reset)
        days_since_maintenance1.append(days_count)
    days_count = 0
    for reset in maintenance.loc[machine]["comp2"]:
        days_count = (days_count+1)*(1-reset)
        days_since_maintenance2.append(days_count)
    days_count = 0
    for reset in maintenance.loc[machine]["comp3"]:
        days_count = (days_count+1)*(1-reset)
        days_since_maintenance3.append(days_count)
    days_count = 0
    for reset in maintenance.loc[machine]["comp4"]:
        days_count = (days_count+1)*(1-reset)
        days_since_maintenance4.append(days_count)
        
maintenance["comp1_delay"] = days_since_maintenance1
maintenance["comp2_delay"] = days_since_maintenance2
maintenance["comp3_delay"] = days_since_maintenance3
maintenance["comp4_delay"] = days_since_maintenance4

Avec une jointure on donne les erreurs aux bonnes dates pour chaque machine, et on fait une somme sur 24h

In [6]:
maintenance.head(10)

comp1  comp2  comp3  comp4  comp1_delay  \
machineID datetime                                                       
1         2014-06-01 06:00:00    0.0    1.0    0.0    0.0          1.0   
          2014-06-02 06:00:00    0.0    0.0    0.0    0.0          2.0   
          2014-06-03 06:00:00    0.0    0.0    0.0    0.0          3.0   
          2014-06-04 06:00:00    0.0    0.0    0.0    0.0          4.0   
          2014-06-05 06:00:00    0.0    0.0    0.0    0.0          5.0   
          2014-06-06 06:00:00    0.0    0.0    0.0    0.0          6.0   
          2014-06-07 06:00:00    0.0    0.0    0.0    0.0          7.0   
          2014-06-08 06:00:00    0.0    0.0    0.0    0.0          8.0   
          2014-06-09 06:00:00    0.0    0.0    0.0    0.0          9.0   
          2014-06-10 06:00:00    0.0    0.0    0.0    0.0         10.0   

                               comp2_delay  comp3_delay  comp4_delay  
machineID datetime                                                    
1         2014-06-01 06:00:00          0.0          1.0          1.0  
          2014-06-02 06:00:00          1.0          2.0          2.0  
          2014-06-03 06:00:00          2.0          3.0          3.0  
          2014-06-04 06:00:00          3.0          4.0          4.0  
          2014-06-05 06:00:00          4.0          5.0          5.0  
          2014-06-06 06:00:00          5.0          6.0          6.0  
          2014-06-07 06:00:00          6.0          7.0          7.0  
          2014-06-08 06:00:00          7.0          8.0          8.0  
          2014-06-09 06:00:00          8.0          9.0          9.0  
          2014-06-10 06:00:00          9.0         10.0         10.0

In [7]:
maintenance.tail(10)

comp1  comp2  comp3  comp4  comp1_delay  \
machineID datetime                                                       
100       2015-12-23 06:00:00    0.0    0.0    0.0    0.0         74.0   
          2015-12-24 06:00:00    0.0    1.0    0.0    0.0         75.0   
          2015-12-25 06:00:00    0.0    0.0    0.0    0.0         76.0   
          2015-12-26 06:00:00    0.0    0.0    0.0    0.0         77.0   
          2015-12-27 06:00:00    0.0    0.0    0.0    0.0         78.0   
          2015-12-28 06:00:00    0.0    0.0    0.0    0.0         79.0   
          2015-12-29 06:00:00    0.0    0.0    0.0    0.0         80.0   
          2015-12-30 06:00:00    0.0    0.0    0.0    0.0         81.0   
          2015-12-31 06:00:00    0.0    0.0    0.0    0.0         82.0   
          2016-01-01 06:00:00    0.0    0.0    0.0    0.0         83.0   

                               comp2_delay  comp3_delay  comp4_delay  
machineID datetime                                                    
100       2015-12-23 06:00:00         14.0         74.0         44.0  
          2015-12-24 06:00:00          0.0         75.0         45.0  
          2015-12-25 06:00:00          1.0         76.0         46.0  
          2015-12-26 06:00:00          2.0         77.0         47.0  
          2015-12-27 06:00:00          3.0         78.0         48.0  
          2015-12-28 06:00:00          4.0         79.0         49.0  
          2015-12-29 06:00:00          5.0         80.0         50.0  
          2015-12-30 06:00:00          6.0         81.0         51.0  
          2015-12-31 06:00:00          7.0         82.0         52.0  
          2016-01-01 06:00:00          8.0         83.0         53.0